In [ ]:
#GENERATE SPECTROGRAMS FOR TRAINING

import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageEnhance
import io

# Parameters
n_mels = 256   # Number of Mel bands
n_fft = 4096  # FFT size
hop_length = 512
fmax = 8000
img_size = (224, 224)  # Size for EfficientNet/ResNet
brightness_factor = 0.8  # Factor to adjust brightness
contrast_factor = 2.0  # Factor to adjust contrast

# Choose your colormap here
colormap = 'viridis'  # Options: 'viridis', 'inferno', 'magma', 'cividis', 'plasma'

def create_spectrogram(filename, save_path, cmap):
    # Load audio file
    y, sr = librosa.load(filename, sr=None)
    
    # Generate mel-spectrogram
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, n_mels=n_mels, fmax=fmax, hop_length=hop_length)
    S_DB = librosa.power_to_db(S, ref=np.max, amin=1e-10, top_db=80)
    
    # Create figure for the spectrogram
    fig, ax = plt.subplots(figsize=(3.5, 3.5))
    librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel', fmax=fmax, cmap=cmap, ax=ax, vmin=S_DB.max() - 80, vmax=S_DB.max())
    ax.axis('off')  # Remove axes
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Adjust margins to fill the figure

    # Save the figure to a buffer
    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi=100, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    buf.seek(0)

    # Open the image and convert to RGB
    img = Image.open(buf).convert('RGB')
    img = img.resize(img_size)

    # Adjust brightness and contrast
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(brightness_factor)
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(contrast_factor)

    # Save the adjusted image
    img.save(save_path)

def process_directory(base_dir, sub_dirs, cmap):
    for sub_dir in sub_dirs:
        current_dir = os.path.join(base_dir, sub_dir)
        save_dir = os.path.join(base_dir, 'spectrograms', sub_dir)
        os.makedirs(save_dir, exist_ok=True)
        
        for filename in os.listdir(current_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(current_dir, filename)
                save_path = os.path.join(save_dir, os.path.splitext(filename)[0] + '.png')
                create_spectrogram(file_path, save_path, cmap)
                print(f"Processed {filename}")

# Your directory setup
base_dir = "path_to_training_data"
sub_dirs = ["positive", "negative"]

# Process directory with chosen colormap
process_directory(base_dir, sub_dirs, colormap)


In [ ]:
#Hold out data for testing 

import os
import shutil
import random
from pathlib import Path

def split_dataset(src_dir, test_size=0.05):
    # Convert to Path object for easier handling
    src_path = Path(src_dir)
    
    # Create test directory at same level as source directory
    test_path = src_path.parent / 'test'
    
    # Create positive and negative directories in test folder
    for class_name in ['positive', 'negative']:
        test_class_path = test_path / class_name
        test_class_path.mkdir(parents=True, exist_ok=True)
        
        # Get list of all files in source class directory
        src_class_path = src_path / class_name
        all_files = list(src_class_path.glob('*'))
        
        # Calculate number of files to move to test
        n_test = int(len(all_files) * test_size)
        
        # Randomly select files for test set
        test_files = random.sample(all_files, n_test)
        
        # Move files to test directory
        for file_path in test_files:
            shutil.move(str(file_path), str(test_class_path / file_path.name))
        
        print(f"Moved {n_test} files from {class_name} to test set")

if __name__ == "__main__":
    # Set random seed for reproducibility
    random.seed(42)
    
    # Set source directory
    src_dir = r"C:\Users\calla\Dropbox\2024\powerfulowl\training_data_post_cluster\spectrograms"
    
    # Split the dataset
    split_dataset(src_dir)

In [ ]:
#TRAIN MODEL

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def create_mobilenet():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    # Freeze all layers except the last few for fine-tuning
    for layer in base_model.layers[:-30]:  # Less layers to freeze since MobileNetV2 is smaller
        layer.trainable = False
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)  # Reduced the size of the dense layer
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)  # Reduced dropout rate
    outputs = Dense(1, activation='sigmoid')(x)
    return Model(inputs=base_model.input, outputs=outputs)

# Create and compile the model
model = create_mobilenet()
model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

# Setup Data Generators with appropriate data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,  # Increased rotation
    width_shift_range=0.1,  # Increased shift
    height_shift_range=0.1,  # Increased shift
    brightness_range=(0.8, 1.2),  # Same brightness adjustment
    shear_range=0.05,  # Added shear transformation
    zoom_range=0.1,  # Added zoom
    fill_mode='nearest',
    validation_split=0.2
)

base_dir = 'C:\\Users\\calla\\Dropbox\\2024\\powerfulowl\\training_data_post_cluster_v2\\spectrograms'
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',
    subset='validation'
)

# Set up callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
]

# Start training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50,
    callbacks=callbacks
)


In [ ]:
# Save the model
model.save('C:\\Users\\calla\\Dropbox\\2024\\powerfulowl\\training_data_v1\\models\\mobilenet_post_clusterv8.keras')


In [ ]:
#test_model

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt

def evaluate_model(model_path, test_data_path, save_path):
    # Load the saved model
    model = tf.keras.models.load_model(model_path)
    
    # Setup test data generator
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    test_generator = test_datagen.flow_from_directory(
        test_data_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        shuffle=False
    )
    
    # Get predictions
    predictions = model.predict(test_generator)
    predicted_classes = (predictions > 0.5).astype(int)
    true_classes = test_generator.classes
    
    # Calculate and print metrics
    print("\nClassification Report:")
    print(classification_report(true_classes, predicted_classes))
    
    # Calculate confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    print("\nConfusion Matrix:")
    print(cm)
    
    # Calculate metrics from confusion matrix
    tn, fp, fn, tp = cm.ravel()
    
    # Calculate accuracy correctly
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    print(f"\nDetailed Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(true_classes, predictions)
    roc_auc = auc(fpr, tpr)
    
    # Plot ROC curve
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    
    # Save the plot instead of displaying it
    plt.savefig(save_path)
    plt.close()
    
    print(f"\nAUC-ROC Score: {roc_auc:.4f}")
    print(f"ROC curve plot saved to: {save_path}")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'auc_roc': roc_auc,
        'confusion_matrix': cm,
        'fpr': fpr,
        'tpr': tpr
    }

# Paths to your model and test data
model_path = 'C:\\Users\\calla\\Dropbox\\2024\\powerfulowl\\training_data_v1\\models\\mobilenet_post_clusterv8.keras'
test_data_path = 'C:\\Users\\calla\\Dropbox\\2024\\powerfulowl\\training_data_post_cluster\\test'
save_path = 'C:\\Users\\calla\\Dropbox\\2024\\powerfulowl\\training_data_v1\\models\\roc_curve8.png'

# Run evaluation
results = evaluate_model(model_path, test_data_path, save_path)

In [ ]:
#Infer on hour long audio files

import os
import librosa
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image, ImageEnhance
import csv
import matplotlib.pyplot as plt
import librosa.display
import gc
import logging
import traceback
import concurrent.futures
import threading
import matplotlib

matplotlib.use('Agg')

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Global variables
model = None
target_size = (224, 224)
brightness_factor = 0.8
contrast_factor = 2.0
batch_size = 32
duration = 5
overlap = 2.5
sample_rate = 48000

# Thread-local storage for model
thread_local = threading.local()

def get_model():
    if not hasattr(thread_local, 'model'):
        thread_local.model = load_model('C:\\Users\\calla\\Dropbox\\2024\\powerfulowl\\training_data_v1\\models\\mobilenet_post_clusterv8.keras')
    return thread_local.model

def create_and_process_spectrogram(y, sr):
    try:
        if len(y) == 0:
            logging.warning("Audio segment is empty")
            return None

        S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, n_mels=256, fmax=24000, hop_length=512)
        S_DB = librosa.power_to_db(S, ref=np.max)

        fig, ax = plt.subplots(figsize=(3.5, 3.5))
        img = ax.imshow(S_DB, aspect='auto', origin='lower', cmap='viridis')
        ax.axis('off')
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

        fig.canvas.draw()
        img_array = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        img_array = img_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        plt.close(fig)

        img = Image.fromarray(img_array).convert('RGB')
        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(brightness_factor)
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(contrast_factor)
        img = img.resize(target_size)

        return img_to_array(img)
    except Exception as e:
        logging.error(f"Error in create_and_process_spectrogram: {str(e)}")
        logging.error(traceback.format_exc())
        return None

def predict_chunk(chunk, sr, start_time):
    try:
        images = []
        segment_times = []
        segment_duration = 5
        segment_samples = int(sr * segment_duration)
        
        for i in range(0, len(chunk), int(sr * (segment_duration - overlap))):
            segment = chunk[i:i + segment_samples]
            if len(segment) < segment_samples:
                break
            segment_time = start_time + i / sr
            segment_times.append(segment_time)
            img_array = create_and_process_spectrogram(segment, sr)
            if img_array is not None:
                images.append(img_array)

        if not images:
            return []

        images = np.array(images)
        images = (images / 255.0) * 2.0 - 1.0
        model = get_model()
        predictions = model.predict(images, batch_size=batch_size)
        results = [(time, pred[0]) for time, pred in zip(segment_times, predictions)]
        
        del images
        gc.collect()
        
        return results
    except Exception as e:
        logging.error(f"Error in predict_chunk at time {start_time}: {str(e)}")
        logging.error(traceback.format_exc())
        return []

def process_file(filename, test_dir):
    file_path = os.path.join(test_dir, filename)
    try:
        logging.info(f"Started processing {filename}")
        chunk_duration = 300  # 5 minutes
        
        if not os.path.isfile(file_path) or not os.access(file_path, os.R_OK):
            logging.error(f"File does not exist or is not readable: {file_path}")
            return

        all_results = []
        start_time = 0

        while True:
            try:
                y, sr = librosa.load(file_path, sr=sample_rate, offset=start_time, duration=chunk_duration)
                if len(y) == 0:
                    break
                chunk_results = predict_chunk(y, sr, start_time)
                all_results.extend([(time, pred) for time, pred in chunk_results])
                logging.info(f"Processed chunk at {start_time} of {filename}, got {len(chunk_results)} results")
                start_time += chunk_duration
            except Exception as e:
                logging.error(f"Error processing chunk at {start_time} of {filename}: {str(e)}")
                break

        if all_results:
            positive_count = sum(1 for _, p in all_results if p > 0.01)
            logging.info(f"Detected {positive_count} potential owl calls out of {len(all_results)} segments in {filename}")
        else:
            logging.warning(f"No results produced for {filename}")

        # Write results to a CSV file for this audio file
        csv_filename = f"{os.path.splitext(filename)[0]}_predictions.csv"
        with open(csv_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Start Time (s)', 'Prediction'])
            for time, prediction in all_results:
                writer.writerow([time, prediction])
        logging.info(f"Wrote {len(all_results)} results to {csv_filename}")

    except Exception as e:
        logging.error(f"Error processing {filename}: {str(e)}")
        logging.error(traceback.format_exc())

def main():
    test_dir = 'J:\\harddrivepowls\\SM3\\sites\\chapelhill\\aprilmay'
    logging.info(f"Looking for WAV files in: {test_dir}")

    if not os.path.exists(test_dir):
        logging.error(f"Directory does not exist: {test_dir}")
        return

    wav_files = [f for f in os.listdir(test_dir) if f.endswith('.wav')]
    
    if not wav_files:
        logging.warning(f"No WAV files found in {test_dir}")
        return

    logging.info(f"Found {len(wav_files)} WAV files: {', '.join(wav_files)}")

    try:
        with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
            futures = [executor.submit(process_file, filename, test_dir) for filename in wav_files]
            concurrent.futures.wait(futures)
        
        logging.info("All files have been processed.")
    except Exception as e:
        logging.error(f"Error in main function: {str(e)}")
        logging.error(traceback.format_exc())

if __name__ == '__main__':
    main()